In [1]:
import numpy as np
from experiment_module import ExperimentModule
from sanchez_ansatz import SanchezAnsatz
from qiskit import transpile
from qiskit_algorithms.optimizers import SPSA
import matplotlib.pyplot as plt
from itertools import product

%matplotlib inline

In [ ]:
def save_plots(em_original: ExperimentModule, em_modified: ExperimentModule, eps: float):
    
    plt.plot(em_original._loss_progression, "--", marker="o", markevery=500, label="Original")
    plt.plot(em_modified._loss_progression, "-.", marker="+", markevery=500, label="Modified")
    plt.ylabel("fid loss")
    plt.xlabel("iterations")
    plt.legend(loc="best")
    plt.savefig(f"results/plot_{em_modified.num_qubits}_{eps}_error.pdf")
    plt.clf()
    plt.close()

In [5]:
qubit_range = [3, 4, 5, 6, 7, 8]
eps_values = [0.5, 0.1, 0.05, 0.01, 0.005, 0.001]

for num_qubits, eps in product(qubit_range, eps_values):

    print("--"*50)
    print(f"\t\t Experiments for {num_qubits} qubits and {eps} error")
    print("--"*50)


    state = np.random.rand(2**num_qubits)
    state = state / np.linalg.norm(state)

    ansatz = SanchezAnsatz(state, eps)

    ## Running original
    print("--"*50)
    print("\t\t Running ORIGINAL Method")
    print("--"*50)
    init_params = ansatz.init_params
    t_ansatz = transpile(ansatz, basis_gates=["cx", "u"])

    em_original = ExperimentModule(
                    t_ansatz,
                    SPSA(maxiter=1000),
                    target_state=state,
                    init_params=init_params
                )

    result_original = em_original.minimize()

    ## Running modified
    print("--"*50)
    print("\t\t Running MODIFIED method")
    print("--"*50)
    ansatz = SanchezAnsatz(state, eps, build_modified=True)

    init_params = ansatz.init_params
    t_ansatz = transpile(ansatz, basis_gates=["cx", "u"])

    em_modified = ExperimentModule(
                    t_ansatz,
                    SPSA(maxiter=1000),
                    target_state=state,
                    init_params=init_params
                )

    result_modified = em_modified.minimize()


----------------------------------------------------------------------------------------------------
		 Running ORIGINAL Method
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
		 Running MODIFIED method
----------------------------------------------------------------------------------------------------


In [6]:
print("Depth ORIGINAL:", em_original._ansatz.depth())
print("Depth Modified:", em_modified._ansatz.depth())

Depth ORIGINAL: 4
Depth Modified: 71
